## Imports and Set up


In [161]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

df = pd.read_csv("../data/flocking_250305.csv")


Using device: cpu


## Prepare Data

In [162]:
# df["NeighborDiffSum"] = df["NeighborDiffSum"].apply(lambda x: eval(x) if isinstance(x, str) else x)
df["NeighborInfo"] = df["NeighborInfo"].apply(lambda x: eval(x) if isinstance(x, str) else x)
df["Angle"] = df["Angle"].astype(float)

"""
group data according to each time step
"""
grouped = df.groupby("Step").agg(list)
# neighbor_diff_sum = np.array(grouped["NeighborDiffSum"].to_list())  # Shape: (num_steps, num_agents, 2)
neighbor_info = np.array(grouped["NeighborInfo"].to_list())  # Shape: (num_steps, num_agents, 2)
angles = np.array(grouped["Angle"].to_list()).reshape(len(grouped), len(grouped.iloc[0]["Angle"]), 1)  # Shape: (num_steps, num_agents, 1)

delta_angle = np.zeros_like(angles)
delta_angle[1:] = angles[1:] - angles[:-1] # Shape: (num_steps, num_agents, 1)

grouped["deltaAngle"] = delta_angle.reshape(len(grouped), -1).tolist()

# X_train, X_test, y_train, y_test = train_test_split(neighbor_diff_sum, delta_angle, test_size=0.2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(neighbor_info, delta_angle, test_size=0.2, random_state=42)

# to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

## Define Model

In [167]:
# define nn
hidden_size_1 = 16
hidden_size_2 = 8
hidden_size_3 = 4

model = nn.Sequential(
    nn.Linear(8, hidden_size_1),
    nn.ReLU(),
    nn.Linear(hidden_size_1, hidden_size_2),
    nn.ReLU(),
    nn.Linear(hidden_size_2, 1)
)

# define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

## Define Model

In [168]:
num_epochs = 1000
for epoch in range(num_epochs):
    optimizer.zero_grad()
    outputs = model(X_train_tensor.view(-1, 8))
    loss = criterion(outputs, y_train_tensor.view(-1, 1))
    loss.backward()
    optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [10/1000], Loss: 0.1095
Epoch [20/1000], Loss: 0.0943
Epoch [30/1000], Loss: 0.0771
Epoch [40/1000], Loss: 0.0692
Epoch [50/1000], Loss: 0.0640
Epoch [60/1000], Loss: 0.0611
Epoch [70/1000], Loss: 0.0592
Epoch [80/1000], Loss: 0.0581
Epoch [90/1000], Loss: 0.0573
Epoch [100/1000], Loss: 0.0568
Epoch [110/1000], Loss: 0.0564
Epoch [120/1000], Loss: 0.0561
Epoch [130/1000], Loss: 0.0558
Epoch [140/1000], Loss: 0.0555
Epoch [150/1000], Loss: 0.0553
Epoch [160/1000], Loss: 0.0550
Epoch [170/1000], Loss: 0.0548
Epoch [180/1000], Loss: 0.0545
Epoch [190/1000], Loss: 0.0543
Epoch [200/1000], Loss: 0.0541
Epoch [210/1000], Loss: 0.0539
Epoch [220/1000], Loss: 0.0537
Epoch [230/1000], Loss: 0.0535
Epoch [240/1000], Loss: 0.0534
Epoch [250/1000], Loss: 0.0532
Epoch [260/1000], Loss: 0.0530
Epoch [270/1000], Loss: 0.0528
Epoch [280/1000], Loss: 0.0526
Epoch [290/1000], Loss: 0.0525
Epoch [300/1000], Loss: 0.0524
Epoch [310/1000], Loss: 0.0523
Epoch [320/1000], Loss: 0.0522
Epoch [330/1000],

## Evaluate the model

In [171]:
with torch.no_grad():
    y_pred = model(X_test_tensor.view(-1, 8))
    test_loss = criterion(y_pred, y_test_tensor.view(-1, 1))
    print(f'Test Loss: {test_loss.item():.4f}')
    print(y_pred)

Test Loss: 0.0406
tensor([[-3.3530e-03],
        [-3.4723e-03],
        [-1.1513e-05],
        ...,
        [-5.1259e-02],
        [ 3.4529e-02],
        [ 5.1091e-02]])


## Save the model

In [170]:
torch.save(model, '../models/20250305_full')